In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

from utils.io import read_parameters

In [ ]:
parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)   

base_folder = parameters["base_folder"]
tracking_data_files = parameters["tracking_data"]

# load data

In [ ]:
tracking_data_df = pd.DataFrame() 
column_dtypes = {'TRACK_ID': 'int16', 
                 'FRAME': 'int16', 
                             'POSITION_X' : 'float16',
                             'POSITION_Y' : 'float16',
                             'POSITION_T' : 'float16'}
print(list(column_dtypes))

for condition in tracking_data_files:
    for filename in tracking_data_files[condition]:
        print(filename)
        data = pd.read_csv(base_folder + "/"+ filename, low_memory=False).drop([0,1,2])
        #print(data.dtypes)
        data_ = data[list(column_dtypes)]
        data_["condition"] = condition
        data_["filename"] = filename
        data_ = data_.astype(column_dtypes)
        print("##################")
        if len(tracking_data_df.index) > 10:
            tracking_data_df = tracking_data_df.append(data_)
        else:
            tracking_data_df = data_.copy()

In [ ]:
tracking_data_df

In [ ]:
for filename in tracking_data_df["filename"].unique():
    tracking_data_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    fig, ax = plt.subplots(figsize=(20,10))
    sns.scatterplot(data = tracking_data_df_, x = "FRAME", y = "TRACK_ID")
    ax.set_title(filename)

In [ ]:
for filename in tracking_data_df["filename"].unique():
    tracking_data_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    fig, ax = plt.subplots(figsize=(20,10))
    tracking_data_df_[["FRAME","TRACK_ID"]].groupby("FRAME").count().plot(ax =ax)
    ax.set_title(filename)

In [ ]:
migration_speed_df = pd.DataFrame(columns=tracking_data_df.columns)


interval = 10
for filename in tracking_data_df["filename"].unique():
    tracks_df_ = tracking_data_df[tracking_data_df["filename"] == filename]
    tracks_df = tracks_df_[["TRACK_ID", "POSITION_X", "POSITION_Y", "POSITION_T", "FRAME"]]
    
    print(filename)
    status = 0
    tracks_num = len(tracks_df["TRACK_ID"].unique())
    
    for track_id in tracks_df["TRACK_ID"].unique():

        single_track_df = tracks_df[tracks_df ["TRACK_ID"]==track_id]
        single_track_df = single_track_df.sort_values(by="FRAME")
        dist = single_track_df.diff(interval).fillna(0.)

        single_track_df["step_size"] = np.sqrt(dist.POSITION_X**2 + dist.POSITION_Y**2)
        single_track_df["step_size_x"] = dist.POSITION_X
        single_track_df["step_size_y"] =  dist.POSITION_Y
        single_track_df["vel"] = np.sqrt(dist.POSITION_X**2 + dist.POSITION_Y**2)/dist.FRAME
        single_track_df["vel_x"] = dist.POSITION_X/dist.FRAME
        single_track_df["vel_y"] =  dist.POSITION_Y/dist.FRAME
        single_track_df["phi"] =  np.arctan2(dist.POSITION_Y,-dist.POSITION_X)*180.0/np.pi

        single_track_df["filename"] = filename
        single_track_df["condition"] = tracks_df_["condition"].iloc[0]
        
        #print("Track ID %s" % track_id)
        #print(single_track_df.head())
        migration_speed_df = migration_speed_df.append(single_track_df, ignore_index=True)
        
        status +=1 
        if status % 500 == 0:
            print("%s out of %s tracks analyzed." % (status,tracks_num))
        
    


In [ ]:
migration_speed_df

In [ ]:
plot_migration_speeds = migration_speed_df.dropna()
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] > interval]

#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(15,10))

sns.lineplot(x = "POSITION_T", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity in microns/min")